In [1]:
import json
import math
import os
import re
import unicodedata
import random
from operator import itemgetter
from pathlib import Path
from pprint import pprint
import pandas as pd
#!pip install --upgrade spacy
import spacy
from spacy.util import compounding, minibatch
from spacy import displacy
# Uncomment if you want Spacy to use GPU for training. Note - this will use transformer architecture
spacy.require_gpu()

True

In [ ]:
nlp = spacy.load("/root/Projects/Atlomy/git/greCy_ATLOMY/training/Lemmatize_transformer_morefiles/lemmatizer/model-best")

In [2]:
nlp = spacy.load("grc_proiel_trf")

# Load Data Table

In [3]:
FILE_PATH = "../assets/NER_assets/Ancient_Words.csv"
# read csv file
df = pd.read_csv(FILE_PATH)

# Preprocess Text

In [4]:
# rename columns to fit code
df.rename(columns = {'Word':'Keyword', 'Category Types':'Label'}, inplace = True)
# If a cell is empty (NaN), Fill it with the value in its parallel "Early" column
for row in df:
    df['Quote'].fillna(df['Early Quote'], inplace=True)
    df['Word Before'].fillna(df['Early Word Before'], inplace=True)
    df['Word After'].fillna(df['Early Word After'], inplace=True)
    df['Label'].fillna(df['Early Category Type'], inplace=True)
# remove rows with no Keyword
df = df.dropna(subset=['Keyword'])
# Remove any row that isn't Greek
pat = '[ء-ي]+'
#df.Keyword.str.contains(pat)
df = df[~df.Keyword.str.contains(pat, na=False)]
#replace new line in df column
df['Keyword'].replace('\n', '', regex=True, inplace=True)
#replace numbers in df
df.replace('\d+', '', regex=True, inplace=True)
#replace hyphens in df column
df.replace('-', '', regex=True, inplace=True)
# replace comma in df column
df['Keyword'].replace(',', '', regex=True, inplace=True)
#replace period in df column
df['Keyword'].replace('\.', '', regex=True, inplace=True)
#replace interpunkt in df column
df['Keyword'].replace('\·', '', regex=True, inplace=True)
# replace multiple spaces in df column
df.replace(' +', ' ', regex=True, inplace=True)
# replace end puntuation spaces in df column
df['Keyword'].replace('\s+$', '', regex=True, inplace=True)


#regular normalization
#for col in df:
#    df[col] = df[col].str.normalize('NFKD')

#removing accents normalization
#for col in df:
#    df[col] = clean_text(df[col].str)


# replace NAN in df column
df = df.fillna(0)
#reset index
df.reset_index(drop=True, inplace=True)

In [5]:
# View DataFrame
df
# if any of the fields "KeyWord", "Quote", "Word Before", "Word After" are "0", drop the row
df = df.drop(df[df.Keyword == 0].index)
df.reset_index(drop=True, inplace=True)
df = df.drop(df[df.Quote == 0].index)
df.reset_index(drop=True, inplace=True)
df = df.drop(df[df['Word Before'] == 0].index)
df.reset_index(drop=True, inplace=True)
df = df.drop(df[df['Word After'] == 0].index)
df.reset_index(drop=True, inplace=True)
df = df.drop(df[df.Label == 0].index)
df.reset_index(drop=True, inplace=True)
df = df.drop(df[df['Lemma'] == 0].index)
df.reset_index(drop=True, inplace=True)

In [6]:
# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from typing import List
from spacy.language import Language
from spacy.tokens import Doc, DocBin, Span
from spacy.util import filter_spans

In [12]:
# This code creates a list of lists of keywords and lines, which is the format required for the keyword and lemma function. It also creates a list of keywords that failed to be processed.
Keywords_and_lines = []
fail = []

for i in range (0,len(df['Keyword'])):
    try:
        Keywords_and_lines.append([df['Keyword'][i], df['Quote'][i], df['Label'][i], df['Word Before'][i], df['Word After'][i], df['Lemma'][i]])
    except Exception:
        fail.append(df['Keyword'][i])
        print('Failed to append ', df['Keyword'][i])


In [13]:
len(Keywords_and_lines)

1533

## preparing Spacy Doc Objects

In [ ]:
linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma: dict = {}
for i, line in enumerate(Keywords_and_lines):
    n = i + 1
    keyword_and_lemma[line[0]] = line[5]
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                print(token.text, line[0], "no match")
    except Exception:
        print("Error on line", n)

In [40]:
print(len(linedocs))
print(len(keyworddocs))
print(len(lemmadocs))
print(len(keyword_and_lemma))

554
1533
1533
771


In [ ]:
import unicodedata

class NormalizeDict:
    def __init__(self, FILE_PATH, norm_method):
        self.FILE_PATH = FILE_PATH
        self.norm_method = norm_method
        self.dendrosearch_lemma_dict = self.load_dendrosearch_lemma_dict(FILE_PATH)
        self.dendrosearch_lemma_dict_normalized = self.normalize_dict(self.dendrosearch_lemma_dict, norm_method)

    def load_dendrosearch_lemma_dict(self, FILE_PATH):
        with open(FILE_PATH, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        # create dict from txt file
        dendrosearch_lemma_dict = {}
        for line in lines:
            key, value = line.split('\t')
            # remove \n from value
            value = value.replace('\n', '')
            dendrosearch_lemma_dict[key] = value
        return dendrosearch_lemma_dict
    
    def normalize_dict(self, dict_to_normalize, norm_method):
        global dict_normalized
        dict_normalized = {}
        for key, value in dict_to_normalize.items():
            try:
                key_normalized = unicodedata.normalize(norm_method, key)
                value_normalized = unicodedata.normalize(norm_method, value)
                dict_normalized[key_normalized] = value_normalized
            except TypeError:
                print("TypeError for key: ", key)
            except ValueError:
                print("ValueError for key: ", key)
            except Exception: # catch all other errors
                print("Error for key: ", key)
        print("Normalized dict contains", len(dict_normalized), "entries")
        #print(dict_normalized)
        return dict_normalized


In [ ]:
dentest = NormalizeDict('../assets/dendrosearch_lemma_dict.txt', 'NFKC')

In [ ]:
linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma: dict = {}
for i, line in enumerate(Keywords_and_lines_normalized_NFKD):
    n = i + 1
    keyword_and_lemma.update({line[0]:line[5]})
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                print(token.text, line[0], "no match")
    except Exception:
        print("Error on line", n)

In [ ]:
linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma_NFKC: dict = {}
for i, line in enumerate(Keywords_and_lines_normalized_NFKC):
    n = i + 1
    keyword_and_lemma_NFKC.update({line[0]:line[5]})
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                print(token.text, line[0], "no match")
    except Exception:
        print("Error on line", n)

In [ ]:
linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma_old: dict = {}
for i, line in enumerate(Keywords_and_lines_normalized_NFKC):
    n = i + 1
    keyword_and_lemma_old.update({line[0]:line[5]})
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                print(token.text, line[0], "no match")
    except Exception:
        print("Error on line", n)

In [ ]:
# load txt dict file from Assets folder
path = '../corpus/INCEpTION_Data'
# read txt file
# merge all .txt files that are in the folder with keyword_and_lemma
INCEpTION_lemma_dict = {}
for file in os.listdir(path):
    if file.endswith(".txt"):
        print(file)
        with open(path + "/" + file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        # create dict from txt file
            for line in lines:
                key, value = line.split('\t')
                # remove \n from value
                value = value.replace('\n', '')
                INCEpTION_lemma_dict[key] = value

# check length of dict
print(INCEpTION_lemma_dict)

In [ ]:
# merge all files into keyword_and_lemma
full_lemma_dict = {**dendrosearch_lemma_dict, **keyword_and_lemma, **INCEpTION_lemma_dict}

In [ ]:
len(full_lemma_dict)

In [ ]:
# check error in dict
clean_full_lemma_dict = {}
for key, value in full_lemma_dict.items():
    if not isinstance(value, str):
        print(key, value, "value is not a string")
        #replace value with key and add to dict
        value = key
        clean_full_lemma_dict[key] = value
        print(key, value, "replaced with", key, value)
    if not isinstance(key, str):
        print(key, value, "key is not a string")
        print(type(key), type(value))
        #replace key with value and add to dict
        key = value
        clean_full_lemma_dict[key] = value
        print(type(key), type(value))
        print(key, value, "replaced with", key, value)
    else:
        clean_full_lemma_dict[key] = value
        


In [ ]:
keywords_list = []
lines_list = []
labels_list = []
word_before_list = []
word_after_list = []
fail = []
lemma_list = []

for i in range (0,len(df['Keyword'])):
    keywords_list.append(df['Keyword'][i])
    lines_list.append(df['Quote'][i])
    labels_list.append(df['Label'][i])
    word_before_list.append(df['Word Before'][i])
    word_after_list.append(df['Word After'][i])
    lemma_list.append(df['Lemma'][i])
else:
    fail.append(df['Keyword'][i])

# Create a list of lists of keywords and lines
Keywords_and_lines = list(zip(keywords_list, lines_list, labels_list, word_before_list, word_after_list, lemma_list))

In [ ]:
# print items with value "0" in Keywords_and_lines
for i in Keywords_and_lines:
    if i[5] == 0:
        print(i[0], i[1], i[2], i[3], i[4], i[5])

In [ ]:
# print lines with int in word_before:
for i in Keywords_and_lines:
    if isinstance(i[3], int):
        print(i[3], i[0], i[1])

In [ ]:
Keywords_and_lines[10]

In [ ]:
type(Keywords_and_lines)

In [ ]:
# normalize nfkd all items in list
Keywords_and_lines = [(unicodedata.normalize('NFKD', keyword), unicodedata.normalize('NFKD', line), label, word_before, word_after, lemma) for keyword, line, label, word_before, word_after, lemma in Keywords_and_lines]


In [ ]:
Keywords_and_lines[10]

In [ ]:
# normalize nfkc all items in list
Keywords_and_lines = [(unicodedata.normalize('NFKC', keyword), unicodedata.normalize('NFKC'), line), label, word_before, word_after, lemma) for keyword, line, label, word_before, word_after, lemma in Keywords_and_lines]


## NFKC Preprocess dataset

In [ ]:
# Create lists from dataframe columns

keywords_list = []
lines_list = []
labels_list = []
word_before_list = []
word_after_list = []
fail = []
lemma_list = []

for i in range (0,len(df['Keyword'])):
    keywords_list.append(df['Keyword'][i])
    lines_list.append(df['Quote'][i])
    labels_list.append(df['Label'][i])
    word_before_list.append(df['Word Before'][i])
    word_after_list.append(df['Word After'][i])
    lemma_list.append(df['Lemma'][i])
else:
    fail.append(df['Keyword'][i])

# Create a list of lists of keywords and lines
Keywords_and_lines = list(zip(keywords_list, lines_list, labels_list, word_before_list, word_after_list, lemma_list))

linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma: dict = {}
for i, line in enumerate(Keywords_and_lines):
    n = i + 1
    keyword_and_lemma.update({line[0]:line[5]})
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                #print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                #print(token.text, line[0], "no match")
                continue
    except Exception:
        print("Error on line", n)
        
        
# merge duplicate linedocs docs that have the same text
def merge_docs(docs: List[Doc]) -> List[Doc]:
    """Merge duplicate docs that have the same text."""
    merged_docs = []
    seen_texts = set()
    for doc in docs:
        if doc.text not in seen_texts:
            seen_texts.add(doc.text)
            merged_docs.append(doc)
    return merged_docs

merged_linedocs_lemmas=(merge_docs(linedocs))

for token in merged_linedocs_lemmas[0]:
    print(token.text, token.lemma_)
for key, value in keyword_and_lemma.items():
    if not isinstance(value, str):
        print(key, value, "is not a string")

#for i in merged_linedocs_lemmas:
#    for token in i:
#        for key, value in keyword_and_lemma.items():
#            if token.text == key:
#                print (token.text, token.lemma_, key, "yes")
            #else: 
            #    print(token.text, key, "no")
            
# load txt dict file from Assets folder
FILE_PATH = "../assets/dendrosearch_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
dendrosearch_lemma_dict = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    dendrosearch_lemma_dict[key] = value

# check length of dict
len(dendrosearch_lemma_dict)

# normalize nfkd for dict
unic_lemma_dict = dendrosearch_lemma_dict
#unic_lemma_dict = {unicodedata.normalize('NFKD', key): unicodedata.normalize('NFKD', value) for key, value in unic_lemma_dict.items()}
unic_lemma_dict = dict(unic_lemma_dict)
#unic_lemma_dict = {key: value for key, value in unic_lemma_dict.items()}

len(unic_lemma_dict)
# merge all files into keyword_and_lemma
#coda_lemma_dict = {#**dendrosearch_lemma_dict, **keyword_and_lemma}
coda_lemma_dict = keyword_and_lemma
# check error in dict
clean_coda_lemma_dict = {}
for key, value in coda_lemma_dict.items():
    if not isinstance(value, str):
        print(key, value, "value is not a string")
        #replace value with key and add to dict
        value = key
        clean_coda_lemma_dict[key] = value
        print(key, value, "replaced with", key, value)
    if not isinstance(key, str):
        print(key, value, "key is not a string")
        print(type(key), type(value))
        #replace key with value and add to dict
        key = value
        clean_coda_lemma_dict[key] = value
        print(type(key), type(value))
        print(key, value, "replaced with", key, value)
    else:
        clean_coda_lemma_dict[key] = value

len(clean_coda_lemma_dict)
# check bool
for key, value in clean_coda_lemma_dict.items():
    if not isinstance(key, str):
        print(key, value, "value is not a string")
        # remove key
        #del full_lemma_dict[key]

# save full_lemma_dict as txt file
with open('../assets/coda_lemma_dict.txt', 'w', encoding='utf-8') as f:
    for key, value in clean_coda_lemma_dict.items():
        f.write(key + "\t" + value + "\n")
f.close()

#load txt dict file from Assets folder
FILE_PATH = "../assets/coda_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
full_lemma_dict1 = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    full_lemma_dict1[key] = value

db = DocBin()
for doc in merged_linedocs_lemmas:
    db.add(doc)
db.to_disk("../corpus/merged_dataset_lemmas.spacy")

Merged_dataset = DocBin().from_disk('../corpus/merged_dataset_lemmas.spacy')
print(len(Merged_dataset))

merged_docbin = DocBin().from_disk("../corpus/merged_dataset_lemmas.spacy")
# get docs from new_docbin
merged_docbin_docs = list(merged_docbin.get_docs(nlp.vocab))

# replace lemmas in merged_linedocs with lemmas from full_lemma_dict. Count replaced lemmas
lemma_count = 0
for doc in merged_docbin_docs:
    for token in doc:
        for key, value in clean_coda_lemma_dict.items():
            if token.text == key:
                print(token.text, token.lemma_, key, value)
                token.lemma_ = value
                lemma_count += 1
print("Total number of lemmas replaced:", lemma_count)

# save whole lemma dataset for evaluation
full_lemmas_dataset = DocBin(docs=merged_docbin_docs)
full_lemmas_dataset.to_disk("../corpus/full_lemmas_dataset.spacy")



## NFKD dataset preprocess

In [ ]:
# Create lists from dataframe columns

keywords_list = []
lines_list = []
labels_list = []
word_before_list = []
word_after_list = []
fail = []
lemma_list = []

for i in range (0,len(df['Keyword'])):
    keywords_list.append(df['Keyword'][i])
    lines_list.append(df['Quote'][i])
    labels_list.append(df['Label'][i])
    word_before_list.append(df['Word Before'][i])
    word_after_list.append(df['Word After'][i])
    lemma_list.append(df['Lemma'][i])
else:
    fail.append(df['Keyword'][i])

# Create a list of lists of keywords and lines
Keywords_and_lines = list(zip(keywords_list, lines_list, labels_list, word_before_list, word_after_list, lemma_list))

linedocs: List[Doc] = []
keyworddocs: List[str] = []
lemmadocs: List[str] = []
keyword_and_lemma: dict = {}
for i, line in enumerate(Keywords_and_lines):
    n = i + 1
    keyword_and_lemma.update({line[0]:line[5]})
    try:
        keyworddocs.append(line[0])
        lemmadocs.append(line[5])
        linedoc = nlp(line[1])
        for token in linedoc:
            if token.text == line[0]:
                #print(token.text, line[0], token.lemma_, line[5], "!!!found match!!!")
                token.lemma_ = line[5]
                if linedoc in linedocs:
                    print(linedoc, "already in linedocs")
                else:
                    linedocs.append(linedoc)
            else:
                #print(token.text, line[0], "no match")
                continue
    except Exception:
        print("Error on line", n)
        
        
# merge duplicate linedocs docs that have the same text
def merge_docs(docs: List[Doc]) -> List[Doc]:
    """Merge duplicate docs that have the same text."""
    merged_docs = []
    seen_texts = set()
    for doc in docs:
        if doc.text not in seen_texts:
            seen_texts.add(doc.text)
            merged_docs.append(doc)
    return merged_docs

merged_linedocs_lemmas=(merge_docs(linedocs))

for token in merged_linedocs_lemmas[0]:
    print(token.text, token.lemma_)
for key, value in keyword_and_lemma.items():
    if not isinstance(value, str):
        print(key, value, "is not a string")

#for i in merged_linedocs_lemmas:
#    for token in i:
#        for key, value in keyword_and_lemma.items():
#            if token.text == key:
#                print (token.text, token.lemma_, key, "yes")
            #else: 
            #    print(token.text, key, "no")
            
# load txt dict file from Assets folder
FILE_PATH = "../assets/dendrosearch_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
dendrosearch_lemma_dict = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    dendrosearch_lemma_dict[key] = value

# check length of dict
len(dendrosearch_lemma_dict)

# normalize nfkd for dict
unic_lemma_dict = dendrosearch_lemma_dict
#unic_lemma_dict = {unicodedata.normalize('NFKD', key): unicodedata.normalize('NFKD', value) for key, value in unic_lemma_dict.items()}
unic_lemma_dict = dict(unic_lemma_dict)
#unic_lemma_dict = {key: value for key, value in unic_lemma_dict.items()}

len(unic_lemma_dict)
# merge all files into keyword_and_lemma
#coda_lemma_dict = {#**dendrosearch_lemma_dict, **keyword_and_lemma}
coda_lemma_dict = keyword_and_lemma
# check error in dict
clean_coda_lemma_dict = {}
for key, value in coda_lemma_dict.items():
    if not isinstance(value, str):
        print(key, value, "value is not a string")
        #replace value with key and add to dict
        value = key
        clean_coda_lemma_dict[key] = value
        print(key, value, "replaced with", key, value)
    if not isinstance(key, str):
        print(key, value, "key is not a string")
        print(type(key), type(value))
        #replace key with value and add to dict
        key = value
        clean_coda_lemma_dict[key] = value
        print(type(key), type(value))
        print(key, value, "replaced with", key, value)
    else:
        clean_coda_lemma_dict[key] = value

len(clean_coda_lemma_dict)
# check bool
for key, value in clean_coda_lemma_dict.items():
    if not isinstance(key, str):
        print(key, value, "value is not a string")
        # remove key
        #del full_lemma_dict[key]

# save full_lemma_dict as txt file
with open('../assets/coda_lemma_dict.txt', 'w', encoding='utf-8') as f:
    for key, value in clean_coda_lemma_dict.items():
        f.write(key + "\t" + value + "\n")
f.close()

#load txt dict file from Assets folder
FILE_PATH = "../assets/coda_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
full_lemma_dict1 = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    full_lemma_dict1[key] = value

db = DocBin()
for doc in merged_linedocs_lemmas:
    db.add(doc)
db.to_disk("../corpus/merged_dataset_lemmas.spacy")

Merged_dataset = DocBin().from_disk('../corpus/merged_dataset_lemmas.spacy')
print(len(Merged_dataset))

merged_docbin = DocBin().from_disk("../corpus/merged_dataset_lemmas.spacy")
# get docs from new_docbin
merged_docbin_docs = list(merged_docbin.get_docs(nlp.vocab))

# replace lemmas in merged_linedocs with lemmas from full_lemma_dict. Count replaced lemmas
lemma_count = 0
for doc in merged_docbin_docs:
    for token in doc:
        for key, value in clean_coda_lemma_dict.items():
            if token.text == key:
                print(token.text, token.lemma_, key, value)
                token.lemma_ = value
                lemma_count += 1
print("Total number of lemmas replaced:", lemma_count)

# save whole lemma dataset for evaluation
full_lemmas_dataset = DocBin(docs=merged_docbin_docs)
full_lemmas_dataset.to_disk("../corpus/full_lemmas_dataset.spacy")



In [ ]:
Keywords_and_lines

In [ ]:
keyword_and_lemma

In [ ]:
coda_lemma_dict

In [ ]:
test_docbin = DocBin().from_disk("../corpus/full_lemmas_dataset.spacy")

In [ ]:
full_lemmas_dataset_docs = list(test_docbin.get_docs(nlp.vocab))
# count the number of lemmas in the dataset
lemma_count = 0
for i in full_lemmas_dataset_docs:
    for token in i:
        lemma_count += 1

In [ ]:
for doc in full_lemmas_dataset_docs:
    for token in doc:
        print(token, token.lemma_)

In [ ]:
len(clean_full_lemma_dict)

In [ ]:
# check bool
for key, value in clean_full_lemma_dict.items():
    if not isinstance(key, str):
        print(key, value, "value is not a string")
        # remove key
        #del full_lemma_dict[key]
        

In [ ]:
# save full_lemma_dict as txt file
with open('../assets/full_lemma_dict.txt', 'w', encoding='utf-8') as f:
    for key, value in clean_full_lemma_dict.items():
        f.write(key + "\t" + value + "\n")
f.close()


In [ ]:
#load txt dict file from Assets folder
FILE_PATH = "../assets/full_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
full_lemma_dict1 = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    full_lemma_dict1[key] = value


In [ ]:
db = DocBin()
for doc in merged_linedocs:
    db.add(doc)
db.to_disk("../corpus/merged_linedocs.spacy")


In [ ]:
db.from_disk("../corpus/merged_linedocs.spacy")
len(db)

In [ ]:
Merged_dataset = DocBin().from_disk('../corpus/merged_linedocs.spacy')
print(len(Merged_dataset))
# merge all .spacy files that are in the folder (except self) with base_db.spacy
for file in os.listdir('../corpus/INCEpTION_Data'):
    if file.endswith(".spacy") and file != 'train.spacy':
        print(file)
        Merged_dataset.merge(DocBin().from_disk('../corpus/INCEpTION_Data/'+file))
print(len(Merged_dataset))
# write the merged file to disk
Merged_dataset.to_disk('../corpus/Merged_dataset.spacy')


In [ ]:
merged_docbin = DocBin().from_disk("../corpus/Merged_dataset.spacy")
# get docs from new_docbin
merged_docbin_docs = list(merged_docbin.get_docs(nlp.vocab))

In [ ]:
# replace lemmas in merged_linedocs with lemmas from full_lemma_dict. Count replaced lemmas
lemma_count = 0
for doc in merged_docbin_docs:
    for token in doc:
        for key, value in clean_full_lemma_dict.items():
            if token.text == key:
                print(token.text, token.lemma_, key, value)
                token.lemma_ = value
                lemma_count += 1
print("Total number of lemmas replaced:", lemma_count)

In [ ]:
# count tokens in lines in linedocs
token_count = 0
for doc in merged_docbin_docs:
    token_count += len(doc)
print("Total number of tokens in linedocs:", token_count)



In [ ]:
# check if a word in dendrosearch_lemma_dict
def is_in_dict(word):
    return word in dendrosearch_lemma_dict

In [ ]:
is_in_dict("ἐκ")

In [ ]:
len(merged_linedocs)

In [ ]:
type(merged_linedocs)

In [ ]:
# save whole lemma dataset for evaluation
full_lemmas_dataset = DocBin(docs=merged_docbin_docs)
full_lemmas_dataset.to_disk("../corpus/train/lemma_train/full_lemmas_dataset.spacy")

In [ ]:
# Split the data in 60:20:20 for train:valid:test dataset
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(merged_docbin_docs, test_size=0.2, random_state=42)
dev_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# without eval, for small datasets (it's faster)
#train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

In [ ]:
# check length of train, dev and test data
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(dev_data)}")
print(f"Number of test examples: {len(test_data)}")


In [ ]:
# look for duplicate lines in merged_docbin_docs
# create list of all lines
all_lines = []
for doc in merged_docbin_docs:
    all_lines.append(doc.text)
    print(doc.text)
# create list of unique lines
unique_lines = []
for line in all_lines:
    if line not in unique_lines:
        unique_lines.append(line)
# check if all lines are unique
if len(all_lines) == len(unique_lines):
    print("All lines are unique")
else:
    print("There are duplicate lines")
    print("Number of all lines:", len(all_lines))
    print("Number of unique lines:", len(unique_lines))
    print("Number of duplicate lines:", len(all_lines)-len(unique_lines))
    print("Percentage of duplicate lines:", (len(all_lines)-len(unique_lines))/len(all_lines))
    print("Percentage of unique lines:", len(unique_lines)/len(all_lines))
    print("Percentage of duplicate lines:", (len(all_lines)-len(unique_lines))/len(all_lines)*100, "%")

 

In [ ]:
# check if same lines are in train, dev and test data
for doc in train_data:
    if doc in dev_data:
        print("doc in dev_data", train_data.index(doc), dev_data.index(doc), dev_data[dev_data.index(doc)].text, " | ", train_data[train_data.index(doc)].text)
    #if doc in test_data:
    #    print("doc in test_data", train_data.index(doc), test_data.index(doc), test_data[dev_data.index(doc)].text, " | ", doc.text)


In [ ]:
# save the train, dev and test data to disk
train_docbin = DocBin(docs=train_data)
train_docbin.to_disk("../corpus/train/lemma_train/lemma_train.spacy")
dev_docbin = DocBin(docs=dev_data)
dev_docbin.to_disk("../corpus/dev/lemma_dev/lemma_dev.spacy")
test_docbin = DocBin(docs=test_data)
test_docbin.to_disk("../corpus/test/lemma_test/lemma_test.spacy")



In [ ]:
import os
from spacy.tokens import DocBin, Doc
# Merge all dataset files in the train folder

Merged_lemma_train_dataset = DocBin().from_disk("../corpus/train/lemma_train/lemma_train.spacy")
print(len(Merged_lemma_train_dataset))
# merge all .spacy files that are in the folder (except self) with base_db.spacy
for file in os.listdir("../corpus/train/lemma_train/"):
    if file.endswith(".spacy") and file != 'lemma_train.spacy':
        print(file)
        Merged_lemma_train_dataset.merge(
            DocBin().from_disk(f"../corpus/train/lemma_train/{file}")
        )
print(len(Merged_lemma_train_dataset))
# write the merged file to disk
Merged_lemma_train_dataset.to_disk("../corpus/train/lemma_train/Merged_lemma_train_dataset.spacy")


In [ ]:
import os
from spacy.tokens import DocBin, Doc
# Merge all dataset files in the dev folder

Merged_lemma_dev_dataset = DocBin().from_disk("../corpus/dev/lemma_dev/lemma_dev.spacy")
print(len(Merged_lemma_dev_dataset))
# merge all .spacy files that are in the folder (except self) with base_db.spacy
for file in os.listdir("../corpus/dev/lemma_dev/"):
    if file.endswith(".spacy") and file != 'lemma_dev.spacy':
        print(file)
        Merged_lemma_dev_dataset.merge(
            DocBin().from_disk(f"../corpus/dev/lemma_dev/{file}")
        )
print(len(Merged_lemma_dev_dataset))
# write the merged file to disk
Merged_lemma_dev_dataset.to_disk("../corpus/dev/lemma_dev/Merged_lemma_dev_dataset.spacy")


In [ ]:
import os
from spacy.tokens import DocBin, Doc
# Merge all dataset files in the dev folder

Merged_lemma_dev_dataset = DocBin().from_disk("../corpus/test/lemma_test/lemma_test.spacy")
print(len(Merged_lemma_dev_dataset))
# merge all .spacy files that are in the folder (except self) with base_db.spacy
for file in os.listdir("../corpus/test/lemma_test/"):
    if file.endswith(".spacy") and file != 'lemma_test.spacy':
        print(file)
        Merged_lemma_dev_dataset.merge(
            DocBin().from_disk(f"../corpus/test/lemma_test/{file}")
        )
print(len(Merged_lemma_dev_dataset))
# write the merged file to disk
Merged_lemma_dev_dataset.to_disk("../corpus/test/lemma_test/Merged_lemma_test_dataset.spacy")


# Training

In [ ]:
# Use Weights and Biases platform to log the model training
import wandb
wandb.init(project="Lemmatizer (Ancient Greek)", entity="atlomy-nlp")

# Training

## Fine-tune Training of Jacobo grc_ud_proiel_trf

## Sentencizer training

In [ ]:
!python -m spacy train ../configs/senter.cfg --output ../training/senter --gpu-id 0 --paths.train ../corpus/train/grc_perseus-ud-train.spacy --paths.dev ../corpus/dev/grc_perseus-ud-dev.spacy --nlp.lang=grc

## Lemmatizer training

In [ ]:
!python -m spacy debug config ../configs/lemmatizer_trf.cfg  --paths.train ../corpus/train/lemma_train/Merged_lemma_train_dataset.spacy --paths.dev ../corpus/dev/lemma_dev/Merged_lemma_dev_dataset.spacy --nlp.lang=grc

In [ ]:
!python -m spacy train ../configs/lemmatizer_trf.cfg  --output ../training/transformer/lemmatizer --paths.train ../corpus/train/lemma_train/Merged_lemma_train_dataset.spacy --paths.dev ../corpus/dev/lemma_dev/Merged_lemma_dev_dataset.spacy --gpu-id 0 --nlp.lang=grc

In [ ]:
!python -m spacy benchmark accuracy ../training/transformer/lemmatizer/model-best/ ../corpus/test/lemma_test/Merged_lemma_test_dataset.spacy --gpu-id 0 

In [ ]:
# Training the transformer with attributes

!python -m spacy debug config ../configs/transformer.cfg --paths.train corpus/train/grc_proiel-ud-train.spacy --paths.dev corpus/dev/grc_proiel-ud-dev.spacy --nlp.lang=grc

In [ ]:
# Training the transformer with attributes
!python -m spacy debug data ../configs/transformer.cfg --paths.train corpus/train/grc_proiel-ud-train.spacy --paths.dev corpus/dev/grc_proiel-ud-dev.spacy --nlp.lang=grc

## POS and other attributes training

# Testing

In [ ]:
# training on seperate dataset files
!python -m spacy train ../configs/lemmatizer_trf.cfg --output ../training/transformer_mostfiles_clean_merged/lemmatizer --gpu-id 0 --paths.train ../corpus/train/lemma_train/clean --paths.dev ../corpus/dev/lemma_dev/clean --nlp.lang=grc


In [ ]:
# training on a merged dataset file
!python -m spacy train ../configs/lemmatizer_trf.cfg --output ../training/transformer_mostfiles_clean_merged/lemmatizer --gpu-id 0 --paths.train ../corpus/train/lemma_train/clean/Merged_lemma_train_clean_dataset.spacy --paths.dev ../corpus/dev/lemma_dev/clean/Merged_lemma_dev_clean_dataset.spacy --nlp.lang=grc


In [ ]:
!python -m spacy benchmark accuracy ../training/transformer_mostfiles_clean/lemmatizer/model-best/ ../corpus-original-with-accents/test/lemma_test/clean/Merged_lemma_test_clean_dataset.spacy --gpu-id 0 

In [ ]:
!python -m spacy benchmark accuracy ../training/Lemmatize_transformer_morefiles/lemmatizer/model-best ../corpus-original-with-accents/test/lemma_test/lemma_test.spacy --gpu-id 0 

In [ ]:
!python -m spacy benchmark accuracy ../training/transformer_mostfiles_clean_merged/lemmatizer/model-best/ ../corpus/test/lemma_test/clean/Merged_lemma_test_clean_dataset.spacy --gpu-id 0 

In [ ]:
!python -m spacy benchmark accuracy ../training/Lemmatize_transformer_morefiles/lemmatizer/model-best ../corpus/test/lemma_test/lemma_test.spacy --gpu-id 0 

In [ ]:
# check if a word in full_lemma_dict
def is_in_dict(word):
    return word in full_lemma_dict

In [ ]:
lemma_nlp = spacy.load("./Models/grc_ud_proiel_trf/grc_ud_proiel_trf-edittree-lemmatizer/model-best")

In [ ]:
# load the test data
#test_docbin = DocBin().from_disk("../corpus/lemma/lemma_test.spacy")
test_docbin_clean = DocBin().from_disk("../corpus/test/lemma_test/clean/Merged_lemma_test_clean_dataset.spacy")
test_docbin_clean_merged = DocBin().from_disk("../corpus/test/lemma_test/clean/Merged_lemma_test_clean_dataset.spacy")
# get docs from new_docbin
#test_docbin_docs = list(test_docbin.get_docs(lemma_only_nlp.vocab))


#text_examples = [doc.text for doc in test_docbin_docs]
clean_test_examples = [doc.text for doc in test_docbin_clean.get_docs(lemma_nlp_clean.vocab)]
clean_merged_test_examples = [doc.text for doc in test_docbin_clean_merged.get_docs(lemma_nlp_clean_merged.vocab)]

In [ ]:
for text in text_examples:
    print("start:", text)

In [ ]:
for doc in test_docbin_docs:
    print("start:", doc.text)

In [ ]:
for token in test_docbin_docs[0]:
    print(token.text, token.lemma_)

In [ ]:
for text in text_examples:
    doc = lemma_nlp(text)
    for token in doc:
        print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))

In [ ]:
for text in text_examples:
    doc = lemma_only_nlp(text)
    for token in doc:
        print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))

In [ ]:
# compare the lemmas of the two models side by side
for text in text_examples:
    print(text)
    doc = lemma_nlp(text)
    doc2 = lemma_only_nlp(text)
    for token, token2 in zip(doc, doc2):
        #check if the lemmas are the same
        if token.lemma_ != token2.lemma_:
            print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text), "||", token2.lemma_)
        else:
            print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))



In [ ]:
# compare the lemmas of the two models side by side
for text in clean_merged_test_examples:
    doc = lemma_nlp_clean(text)
    doc2 = lemma_nlp_clean_merged(text)
    #doc3 = lemma_nlp(text)
    for token, token2 in zip(doc, doc2):
        #check if the lemmas are the same
        if token.lemma_ != token2.lemma_:
            print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_)
            #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
        else:
            #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
            pass

In [ ]:
# compare the lemmas of the two models side by side
for text in text_examples:
    doc = lemma_nlp(text)
    doc2 = lemma_only_nlp(text)
    doc3 = lemma_nlp(text)
    for token, token2, token3 in zip(doc, doc2, doc3):
        #check if the lemmas are the same
        if token.lemma_ != token2.lemma_:
            print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_,  "|", "3: ", token3.lemma_)
            #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
        else:
            #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
            pass

In [ ]:
# compare lemmatization of two models by comparing the number of lemmas that are not in the lemma dict
lemma_count = 0
lemma_only_count = 0
nlp_count = 0
for text in text_examples:
    doc = lemma_nlp(text)
    for token in doc:
        if not is_in_dict(token.text):
            lemma_count += 1
for text in text_examples:
    doc = lemma_only_nlp(text)
    for token in doc:
        if not is_in_dict(token.text):
            lemma_only_count += 1
for text in text_examples:
    doc = nlp(text)
    for token in doc:
        if not is_in_dict(token.text):
            nlp_count += 1
print("Number of lemmas not in lemma dict for lemma model:", lemma_count)
print("Number of lemmas not in lemma dict for lemma only model:", lemma_only_count)
print("Number of lemmas not in lemma dict for origin model:", nlp_count)



In [ ]:
# compare lemmatization of two models by comparing the number of lemmas that are not in the lemma dict
lemma_count = 0
lemma_only_count = 0 
nlp_count = 0
for doc in test_docbin.get_docs(lemma_nlp.vocab):
    for token in doc:
        if not is_in_dict(token.text):
            lemma_count += 1
for doc in test_docbin.get_docs(lemma_only_nlp.vocab):
    for token in doc:
        if not is_in_dict(token.text):
            lemma_only_count += 1
for doc in test_docbin.get_docs(nlp.vocab):
    for token in doc:
        if not is_in_dict(token.text):
            nlp_count += 1
print("Number of lemmas not in lemma dict for lemma model:", lemma_count)
print("Number of lemmas not in lemma dict for lemma only model:", lemma_only_count)
print("Number of lemmas not in lemma dict for origin model:", nlp_count)



In [ ]:
import unicodedata
def clean_text(text: str) -> str:
    #Cleans the given text by stripping accents and lowercasing.
    try:
        non_accent_characters = [
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
        ]
    # str.lower() works for unicode characters
        return ''.join(non_accent_characters).lower()
    except TypeError:
        return text


## Compare accented and non accented models

In [ ]:
lemma_nlp = spacy.load("../training/Lemmatize_transformer_morefiles/lemmatizer/model-best")

In [ ]:
lemma_no_accents_nlp = spacy.load("../training/transformer/lemmatizer/model-best")

In [ ]:
# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from typing import List
from spacy.language import Language
from spacy.tokens import Doc, DocBin, Span
from spacy.util import filter_spans

In [ ]:
# load the test data
#test_docbin = DocBin().from_disk("../corpus/lemma/lemma_test.spacy")
test_docbin = DocBin().from_disk("../corpus_morefiles_model/test/lemma_test/lemma_test.spacy")
test_docbin_no_accents = DocBin().from_disk("../corpus/test/lemma_test/Merged_lemma_test_dataset.spacy")
# get docs from new_docbin
test_docbin_docs = list(test_docbin.get_docs(lemma_nlp.vocab))


#text_examples = [doc.text for doc in test_docbin_docs]
test_examples = [doc.text for doc in test_docbin.get_docs(lemma_nlp.vocab)]
test_examples_no_accents = [doc.text for doc in test_docbin_no_accents.get_docs(lemma_no_accents_nlp.vocab)]
test_docbin_no_accents_docs = list(test_docbin.get_docs(lemma_no_accents_nlp.vocab))


In [ ]:
for text in test_examples:
    print("start:", text)

In [ ]:
for text in test_examples_no_accents:
    print("start:", text)

In [ ]:
# check if a word in full_lemma_dict
def is_in_dict(word):
    return word in full_lemma_dict

In [ ]:
#load txt dict file from Assets folder
FILE_PATH = "../assets/full_lemma_dict.txt"
# read txt file
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# create dict from txt file
full_lemma_dict = {}
for line in lines:
    key, value = line.split('\t')
    # remove \n from value
    value = value.replace('\n', '')
    full_lemma_dict[key] = value


In [ ]:
for token in test_docbin_docs[0]:
    print(token.text, token.lemma_)

In [ ]:
for text in test_examples:
    doc = lemma_nlp(text)
    for token in doc:
        print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))

In [ ]:
for text in test_examples_no_accents:
    doc = lemma_no_accents_nlp(text)
    for token in doc:
        print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))

In [ ]:
# compare the lemmas of the two models side by side
for text in test_examples:
    print(text)
    doc = lemma_nlp(text)
    doc2 = lemma_no_accents_nlp(text)
    for token, token2 in zip(doc, doc2):
        #check if the lemmas are the same
        if token.lemma_ != token2.lemma_:
            print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text), "||", token2.lemma_)
        else:
            print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))



In [ ]:
# compare the lemmas of the two models side by side
for text in test_examples:
    doc = lemma_nlp(clean_text(text))
    doc2 = lemma_no_accents_nlp(clean_text(text))
    #doc3 = lemma_nlp(text)
    for token, token2 in zip(doc, doc2):
        #check if the lemmas are the same
        if token.lemma_ != token2.lemma_:
            print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_)
            #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
        else:
            #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
            pass

In [ ]:
# compare lemmatization of two models by comparing the number of lemmas that are not in the lemma dict
lemma_count = 0
lemma_no_accents_count = 0
for text in test_examples:
    doc = lemma_nlp(text)
    for token in doc:
        if not is_in_dict(token.text):
            lemma_count += 1
for text in test_examples_no_accents:
    doc = lemma_no_accents_nlp(text)
    for token in doc:
        if not is_in_dict(token.text):
            lemma_no_accents_count += 1
print("Number of lemmas not in lemma dict for accents model:", lemma_count)
print("Number of lemmas not in lemma dict for non accents model:", lemma_no_accents_count)



In [ ]:
# compare lemmatization of two models by comparing the number of lemmas that are not in the lemma dict
lemma_count = 0
lemma_no_accents_count = 0 
for doc in test_docbin.get_docs(lemma_nlp.vocab):
    for token in doc:
        if not is_in_dict(token.text):
            lemma_count += 1
for doc in test_docbin_no_accents.get_docs(lemma_no_accents_nlp.vocab):
    for token in doc:
        if not is_in_dict(token.text):
            lemma_no_accents_count += 1
print("Number of lemmas not in lemma dict for lemma model:", lemma_count)
print("Number of lemmas not in lemma dict for lemma only model:", lemma_no_accents_count)



In [ ]:
# remove accents definition
import unicodedata
def clean_text(text: str) -> str:
    #Cleans the given text by stripping accents and lowercasing.
    try:
        non_accent_characters = [
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
        #or char == '̓'  # Greek coronis
        ]
    # str.lower() works for unicode characters
        return ''.join(non_accent_characters).lower()
    except TypeError:
        return text


In [ ]:
#text = "καὶ γὰρ ἀναπνεῖ καὶ ἐκπνεῖ ταύτῃ καὶ ὁ πταρμὸς διὰ ταύτης γίνεται πνεύματος ἀθρόου ἔξοδος σημεῖον οἰωνιστικὸν καὶ ἱερὸν μόνον τῶν πνευμάτων"
text =("Ταῦτ’ οὖν εἴς τε τὸν παρόντα καὶ τὸν ἑξῆς ἅπαντα λόγον οἷον ὑποθέσεις τινὰς τῶν ἀποδείξεων λαμβάνοντες ἐν ἑκάστῳ τῶν ὀργάνων τὴν ἐξ αὐτῶν ὠφέλειαν ἐροῦμεν ἀπὸ τῶν δακτύλων αὖθις ἀρξάμενοι.")
text_no_accents = clean_text("Ταῦτ’ οὖν εἴς τε τὸν παρόντα καὶ τὸν ἑξῆς ἅπαντα λόγον οἷον ὑποθέσεις τινὰς τῶν ἀποδείξεων λαμβάνοντες ἐν ἑκάστῳ τῶν ὀργάνων τὴν ἐξ αὐτῶν ὠφέλειαν ἐροῦμεν ἀπὸ τῶν δακτύλων αὖθις ἀρξάμενοι.")


In [ ]:
# compare lemmatization of two models by comparing the number of lemmas that are not in the lemma dict
lemma_count = 0
lemma_no_accents_count = 0
for text in test_examples:
    doc = lemma_nlp(clean_text(text))
    for token in doc:
        if not is_in_dict(token.text):
            lemma_count += 1
for text in test_examples_no_accents:
    doc = lemma_no_accents_nlp(clean_text(text))
    for token in doc:
        if not is_in_dict(token.text):
            lemma_no_accents_count += 1
print("Number of lemmas not in lemma dict for accents model:", lemma_count)
print("Number of lemmas not in lemma dict for non accents model:", lemma_no_accents_count)



In [ ]:
# compare the lemmas of the two models side by side
doc = lemma_nlp(text)
doc2 = lemma_no_accents_nlp(text_no_accents)
#doc3 = lemma_nlp(text)
for token, token2 in zip(doc, doc2):
    #check if the lemmas are the same
    if token.lemma_ != token2.lemma_:
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_)
        #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
    else:
        #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "==" , "2: ", token2.lemma_)


In [ ]:
# compare the lemmas of the two models side by side
doc = lemma_nlp(text)
doc2 = lemma_no_accents_nlp(text)
#doc3 = lemma_nlp(text)
for token, token2 in zip(doc, doc2):
    #check if the lemmas are the same
    if token.lemma_ != token2.lemma_:
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_)
        #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
    else:
        #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "==" , "2: ", token2.lemma_)


In [ ]:
# compare the lemmas of the two models side by side
doc = lemma_no_accents_nlp(text)
doc2 = lemma_no_accents_nlp(text_no_accents)
#doc3 = lemma_nlp(text)
for token, token2 in zip(doc, doc2):
    #check if the lemmas are the same
    if token.lemma_ != token2.lemma_:
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "!=" , "2: ", token2.lemma_)
        #print("(", text, ")", token.text, ":\n", '\033[1m', "1: ", token.lemma_, "!=" , "2: ", token2.lemma_, '\033[0m',  "|", "3: ", token3.lemma_)
    else:
        #print(token.text, ":", token.lemma_, "|" , "in Lemma dict:", is_in_dict(token.text))
        print("(", text, "):\n", "|", token.text, "|", "1: ", token.lemma_, "==" , "2: ", token2.lemma_)


# Package Model

In [ ]:
nlp = spacy.load('./Models/grc_ud_proiel_trf/NER_S_grc_ud_proiel_trf-new_db_3-accumulate-gradient/model-best')

In [ ]:
import unicodedata as ud

def clean_text(text):
    cleaned = re.sub(r"[\(\[].*?[\)\]]", "", text)
    cleaned = ud.normalize('NFKD', cleaned)
    return cleaned

#Test your text
test_text = input("Enter your testing text: ")
#clean text
test = clean_text(test_text)
#predict
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
for token in doc:
    print(token.text, ' !!! ', token.lemma_, ' !!! ', token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
# testing new pipe with dit tree lemmatizer
import unicodedata as ud

def clean_text(text):
    cleaned = re.sub(r"[\(\[].*?[\)\]]", "", text)
    cleaned = ud.normalize('NFKD', cleaned)
    return cleaned

#Test your text
test_text = input("Enter your testing text: ")
#clean text
test = clean_text(test_text)
#predict
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
for token in doc:
    print(token.text, ' !!! ', token.lemma_, ' !!! ', token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
nlp.pipe_names

In [ ]:
nlp._pipe_configs

In [ ]:
lemmatizer_nlp = spacy.load("./Models/grc_ud_proiel_trf/grc_ud_proiel_trf-edittree-lemmatizer-only/model-best")
lemmatizer_nlp.pipe_names

In [ ]:
nlp.add_pipe('trainable_lemmatizer', source=lemmatizer_nlp, before='lemmatizer')

In [ ]:
nlp.pipe_names

In [ ]:
# move trainable lemmatizer to after the lemmatizer
nlp.remove_pipe('trainable_lemmatizer')
nlp.add_pipe('trainable_lemmatizer', source=lemmatizer_nlp, after='lemmatizer')
nlp.pipe_names

In [ ]:
# testing new pipe with edit tree lemmatizer
import unicodedata as ud

def clean_text(text):
    cleaned = re.sub(r"[\(\[].*?[\)\]]", "", text)
    cleaned = ud.normalize('NFKD', cleaned)
    return cleaned

#Test your text
test_text = input("Enter your testing text: ")
#clean text
test = clean_text(test_text)
#predict
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
for token in doc:
    print(token.text, ' !!! ', token.lemma_, ' !!! ', token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
# package nlp pipeline
nlp.to_disk("./Models/grc_ud_proiel_trf/grc_ud_proiel_trf-NER-Trainable_Lemmatizer") # save model to disk


In [ ]:
!python -m spacy package ./Models/grc_ud_proiel_trf/grc_ud_proiel_trf-NER-Trainable_Lemmatizer ./Models/grc_ud_proiel_trf/grc_ud_proiel_trf-NER-Trainable_Lemmatizer --name grc_ud_proiel_trf-NER-Trainable_Lemmatizer --version 0.0.1

In [ ]:
# remove accents definition
import unicodedata
def clean_text(text: str) -> str:
    #Cleans the given text by stripping accents and lowercasing.
    try:
        non_accent_characters = [
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
        ]
    # str.lower() works for unicode characters
        return ''.join(non_accent_characters).lower()
    except TypeError:
        return text


In [ ]:
from spacy.tokens import DocBin, Doc

test_data= DocBin().from_disk("/root/Projects/Atlomy/git/greCy_ATLOMY/corpus/test/lemma_test/lemma_test.spacy")
test_docbin_docs = list(test_data.get_docs(nlp.vocab))

In [ ]:
# remove accents from text and lemmas and attributes of tokens and save to new docbin
new_test_docbin = DocBin()
for doc in test_docbin_docs:
    new_doc = Doc(nlp.vocab, words=[clean_text(token.text) for token in doc], spaces=[token.whitespace_ for token in doc])
    for new_token, token in zip(new_doc, doc):
        new_token.lemma_ = clean_text(token.lemma_)
        new_token.pos_ = token.pos_
        new_token.tag_ = token.tag_
        new_token.dep_ = token.dep_
        #new_token.shape_ = token.shape_
        #new_token.is_alpha = token.is_alpha
       # new_token.is_stop = token.is_stop
    new_test_docbin.add(new_doc)
#new_test_docbin.to_disk("/root/Projects/Atlomy/git/greCy_ATLOMY/corpus/train/lemma_train/aeschylus_ii_train_clean.spacy")


In [ ]:
for doc in test_docbin_docs:
    for token in doc:
        print(token.text, ' !!! ', token.lemma_, ' !!! ', token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
for doc in new_test_docbin.get_docs(nlp.vocab):
    for token in doc:
        print(token.text, ' !!! ', token.lemma_, ' !!! ', token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
# compare test_docbin_docs with new_test_docbin
for doc1, doc2 in zip(test_docbin_docs, new_test_docbin.get_docs(nlp.vocab)):
    for token1, token2 in zip(doc1, doc2):
        if clean_text(token1.text) == token2.text:
            #print(token1.text, ' !!! ', token2.text)
            if clean_text(token1.lemma_) == token2.lemma_:
                #print(token1.lemma_, ' !!! ', token2.lemma_)
                if token1.pos_ != token2.pos_:
                    print(token1, token2, token1.pos_, ' !!! ', token2.pos_)
                if token1.is_alpha != token2.is_alpha:
                    print(token1, token2, token1.is_alpha, ' !!! ', token2.is_alpha)
                if token1.is_stop != token2.is_stop:
                    print(token1, token2, token1.is_stop, ' !!! ', token2.is_stop)

In [ ]:
# remove accents
def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

In [ ]:
for doc in test_docbin_docs:
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
        print(remove_accents(token.text), remove_accents(token.lemma_), token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
# remove accents from everything in the doc and save a new docbin
new_docbin = DocBin()
for doc in test_docbin_docs:
    new_doc = Doc(nlp.vocab, words=[remove_accents(token.text) for token in doc], spaces=[token.whitespace_ for token in doc])
    for token in new_doc:
        token.lemma_ = remove_accents(token.lemma_)
    new_docbin.add(new_doc)


# Solving preprocess txt issues

In [ ]:
# print docs in new docbin
for doc in new_docbin.get_docs(nlp.vocab):
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
# Solving text issues
df.loc[df['Lemma'] == 'ὁ μέν...ὁ δέ']
# create a dictionary of the words and their lemmas
phrase_dict = {
    'Τὸ': 'ὁ',
    'μὲν': 'μέν',
    'μὲν': 'μέν',
    'δ’': 'δέ',
    'δ': 'δέ',
    'δὲ': 'δέ',
    'τὸν': 'ὁ',
    'τὴν': 'ἡ',
    'τὴ': 'ἡ',
    'τὸ': 'ὁ',
    'αἱ': 'ὁ',
    'Αἱ': 'ὁ',
    'Τὰ': 'ὁ',
    'τὰ': 'ὁ',
    'τὰς': 'ὁ',
    'τὰν': 'ὁ',
}
# create wrds dictionary

#find rows with lemma "ὁ μέν...ὁ δέ"
#df.loc[df['Lemma'] == 'ὁ μέν...ὁ δέ']
# if more than one word, then the lemma is "ὁ μέν...ὁ δέ"
for index, row in df.iterrows():
    if row['Lemma'] == 'ὁ μέν...ὁ δέ':
        # if only one word, then the lemma is the lemma for the word in phrase_dict.
        if len(row['Keyword'].split()) == 1:
            if row['Keyword'] in phrase_dict:
                # replace the lemma with the value in the dictionary
                row['Lemma'] = phrase_dict[row['Keyword']]
                #replace the category type with "Division"
                df.at[index, 'Label'] = 'Division'
            # if word not in dictionary:
            else:
                print('false')
        else:
            # split the keyword to words and add the second word to the df as a row after the first word
            # add the second word to the df as a row after the first word
            # copy all row values to a new row
            df.loc[index+1.5] = [row['Keyword'].split()[1], row['Word Before'], row['Word After'], row['Quote'], row['Label'], row['Lemma'], row['Early Category Type'], row['Early Word'], row['Early Word Before'], row['Early Word After'], row['Early Quote']]
            df.loc[index+1.5, 'Label'] = 'Division'
            df.loc[index+1.5, 'Lemma'] = phrase_dict[df.loc[index+1.5,'Keyword']]
    # replace the first word with the lemma "ὁ μέν"
            print (index, row['Keyword'])
            print(df.loc[index+1.5])
            #df = df.sort_index().reset_index(drop=True)
            #print (df.loc[index+1.5, 'Keyword', index, 'keyword'])
            #print("new word: ", row['Keyword'].split()[0])
            #print("new Lemma: ", phrase_dict[row['Keyword']])
            row['Keyword'] = row['Keyword'].split()[0]
            df.at[index, 'Lemma'] = df.at[index, 'Keyword']
            #replace the category type with "Division"
            df.at[index, 'Label'] = 'Division'
            # replace the first word with the lemma "ὁ δέ"
            #replace the category type with "Division"
            #print (row['Keyword'])
            #print (df.loc[index+1, 'Keyword'])
            #print (df.loc[index+1, 'Lemma'])
            #print ("more: ", row['Keyword'])
df = df.sort_index().reset_index(drop=True)


